##Installing and Importing Libraries


In [ ]:
!pip install faker
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.0 MB/s eta 0:00:00


In [ ]:
import random
import uuid
from faker import Faker
from collections import Counter
import pycountry



##Creatig Enviroment variables and connecting to AWS Database

In [ ]:
from google.colab import userdata
import os
import psycopg2



# Get private dats from collab
DB_USER = userdata.get("DB_USER")
DB_PASSWORD = userdata.get("DB_PASSWORD")
DB_HOST = userdata.get("DB_HOST")
DB_PORT = userdata.get("DB_PORT")
DB_NAME = userdata.get("DB_NAME")

# Datavase Connection
DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

conn = psycopg2.connect(DB_URL)
cursor = conn.cursor()

print("✅ Connected!")


✅ Connected!


## Creating Tables


In [ ]:

# Creating Tables
#SQL QUERY
create_tables_query = """

DROP TABLE IF EXISTS sales;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS retailers;
DROP TABLE IF EXISTS operating_expenses;


-- Tabella retailers (rivenditori)
CREATE TABLE IF NOT EXISTS retailers (
    retailer_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(50) NOT NULL,
    city VARCHAR(50) NOT NULL,
    commission_rate DECIMAL(5,2) CHECK (commission_rate BETWEEN 0 AND 100),
    contact_email VARCHAR(100) UNIQUE
);

-- Tabella customers (clienti)
CREATE TABLE IF NOT EXISTS customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(50) NOT NULL,
    city VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    phone VARCHAR(20)
);

-- Tabella sales (vendite)
CREATE TABLE IF NOT EXISTS sales (
    transaction_id SERIAL PRIMARY KEY,
    sale_date DATE NOT NULL,
    retailer_id INT REFERENCES retailers(retailer_id) ON DELETE CASCADE,
    customer_id INT REFERENCES customers(customer_id) ON DELETE SET NULL,
    product_category VARCHAR(50) NOT NULL,
    product_name VARCHAR(100) NOT NULL,
    serial_number VARCHAR(50) UNIQUE NOT NULL,
    quantity INT CHECK (quantity > 0),
    unit_price DECIMAL(10,2) CHECK (unit_price > 0),
    discount DECIMAL(5,2) CHECK (discount BETWEEN 0 AND 100),
    total_price DECIMAL(10,2) CHECK (total_price > 0)
);

-- Tabella operating_expenses (spese operative)
CREATE TABLE IF NOT EXISTS operating_expenses (
    expense_id SERIAL PRIMARY KEY,
    category VARCHAR(50) NOT NULL,
    amount DECIMAL(10,2) CHECK (amount > 0),
    expense_date DATE NOT NULL,
    description TEXT
);
"""

# Esegui la creazione delle tabelle
cursor.execute(create_tables_query)
conn.commit()

print("✅ TABLES CREATED")





✅ TABLES CREATED


##Retatilers Data

In [ ]:
fake = Faker()

# Reatilers List
retailers = [fake.name() for _ in range(20)]

#Connection
conn = psycopg2.connect(DB_URL)
cursor = conn.cursor()

for retailer in retailers:
    #ID
    retailer_id = retailers.index(retailer) + 1

    #Contact
    contact_email = fake.email()

    #Commisinon and Experience
    commission_rate = round(random.uniform(5, 15), 2)  # Commissione tra il 5% e il 15%
    monthly_sales = round(random.uniform(10000, 500000), 2)  # Fatturato mensile ipotetico
    years_in_business = random.randint(1, 15)


    #Location
    location = fake.location_on_land()
    city = location[2]  # Città
    country = location[3]
    country = pycountry.countries.get(alpha_2 = country).name

#
#SQL QUERY
    cursor.execute("""
  INSERT INTO retailers (
        retailer_id, name, country,
        city, commission_rate, contact_email)
         VALUES (%s, %s, %s, %s, %s, %s)
""", (
    retailer_id, retailer, country,
    city, commission_rate, contact_email,
    ))
conn.commit()

print("✅ Reatailers table was corrected populated")

✅ Reatailers table was corrected populated


Checking if the table was correctly populated



In [ ]:
query = "SELECT * FROM retailers LIMIT 20;"
cursor.execute(query)
result = cursor.fetchall()
for row in result:
  print(row)



(1, 'Bryan Phillips', 'Pakistan', 'Mianwali', Decimal('7.90'), 'raymond88@example.com')
(2, 'Jennifer Adams', 'Russian Federation', 'Yashkino', Decimal('8.79'), 'morrisnicole@example.com')
(3, 'Brian Johnson', 'Japan', 'Okegawa', Decimal('12.11'), 'robert86@example.com')
(4, 'Kim Pena', 'France', 'Quimper', Decimal('5.07'), 'ewatson@example.net')
(5, 'Suzanne Skinner', 'Germany', 'Zeitz', Decimal('14.22'), 'debra74@example.net')
(6, 'Amy Williams', 'Brazil', 'Palmeira', Decimal('8.28'), 'tgilbert@example.org')
(7, 'Marissa Parker', 'Somalia', 'Baki', Decimal('6.21'), 'iholmes@example.net')
(8, 'Kim Landry', 'Montenegro', 'Budva', Decimal('11.62'), 'hollandjessica@example.net')
(9, 'Barbara Ray', 'India', 'Sheoganj', Decimal('12.16'), 'amber83@example.net')
(10, 'Kimberly Armstrong', 'Rwanda', 'Rwamagana', Decimal('6.51'), 'hspears@example.com')
(11, 'Jessica Parker', 'Sri Lanka', 'Dehiwala-Mount Lavinia', Decimal('6.66'), 'wcooper@example.org')
(12, 'Joseph Espinoza', 'Mexico', 'Progre

##Customers Data

In [ ]:
# Generazione dati anagrafici per i clienti
customers = [fake.name() for _ in range(150)]

#Connection
conn = psycopg2.connect(DB_URL)
cursor = conn.cursor()


for customer in customers:
     #ID
    customer_id = customers.index(customer) + 1

    #Contact
    email = fake.email()
    phone = fake.phone_number()[:10]



    #Location
    location = fake.location_on_land()
    city = location[2]  # Città
    country = location[3]
    country = pycountry.countries.get(alpha_2 = country).name


# inserting data into tables
#SQL QUERY
    cursor.execute("""
  INSERT INTO customers (
        customer_id, name, country,
        city, email, phone)
         VALUES (%s, %s, %s, %s, %s, %s)
""", (
    customer_id, customer, country,
    city, email, phone,
    ))
conn.commit()

print("✅ Customers table was corrected populated")

✅ Customers table was corrected populated


Checking if the table was correctly populated



In [ ]:
query = "SELECT * FROM customers LIMIT 20;"
cursor.execute(query)
result = cursor.fetchall()
for row in result:
  print(row)



(1, 'Ashley Jones', 'Libya', 'Ghat', 'trosales@example.com', '602-465-75')
(2, 'Steven Collins', 'China', 'Pingyi', 'mmiles@example.org', '001-972-91')
(3, 'Matthew Ray', 'United States', 'Lino Lakes', 'darellano@example.com', '(727)479-6')
(4, 'Rebecca Ramos', 'Russian Federation', 'Danilov', 'paul62@example.com', '+1-319-897')
(5, 'Phillip Huynh', 'Nicaragua', 'Jinotepe', 'jenkinsmonica@example.net', '373.517.46')
(6, 'Andre Yates', 'India', 'Periyakulam', 'martha12@example.net', '276.455.29')
(7, 'Heather Thomas', 'Chile', 'Ovalle', 'ashleyjoseph@example.org', '(847)563-3')
(8, 'Jill Vance', 'Brazil', 'Resplendor', 'garciafrancisco@example.com', '001-679-32')
(9, 'Jose Carrillo', 'Australia', 'Perth', 'catherine66@example.com', '001-469-61')
(10, 'Fernando Simmons', 'Myanmar', 'Myawadi', 'stevenstammy@example.net', '234-860-67')
(11, 'Grace Freeman', 'Egypt', 'Alexandria', 'mccoymark@example.com', '770-709-99')
(12, 'Kyle Campbell', 'United Kingdom', 'Rutherglen', 'paulwong@example.

##Generating Sales Data as realistic as possible


Querying the databse to extract Retailers and Customers to Generate Sales Table


In [ ]:
#Cusotmers
#SQL QUERY
query = "SELECT DISTINCT(customer_id) FROM customers;"
cursor.execute(query)
customers_id = [row[0] for row in cursor.fetchall()]

#Retailers
#SQL QUERY
query = "SELECT DISTINCT(retailer_id) FROM retailers;"
cursor.execute(query)
retailers_id = [row[0] for row in cursor.fetchall()]

print(customers_id[:5],retailers_id[:5])

[116, 87, 71, 68, 51] [4, 10, 6, 14, 13]


In [ ]:
# Definizione dei prodotti (10 per categoria con prezzi reali)
products = {
    'Cardio': [
        {'name': 'Treadmill', 'price': 2500.00},
        {'name': 'Exercise Bike', 'price': 1200.00},
        {'name': 'Rowing Machine', 'price': 1800.00},
        {'name': 'Elliptical Trainer', 'price': 1500.00},
        {'name': 'Stepper', 'price': 950.00},
        {'name': 'Spin Bike', 'price': 1400.00},
        {'name': 'Pro Treadmill', 'price': 3200.00},
        {'name': 'Professional Rower', 'price': 2800.00},
        {'name': 'Magnetic Bike', 'price': 1100.00},
        {'name': 'Air Bike', 'price': 1600.00}
    ],
    'Strength': [
        {'name': 'Flat Bench', 'price': 700.00},
        {'name': 'Power Rack', 'price': 1200.00},
        {'name': 'Leg Press Machine', 'price': 2000.00},
        {'name': 'Abdominal Machine', 'price': 800.00},
        {'name': 'Adjustable Dumbbells', 'price': 400.00},
        {'name': 'Olympic Barbell', 'price': 350.00},
        {'name': 'Smith Machine', 'price': 2800.00},
        {'name': 'Chest Press Machine', 'price': 2200.00},
        {'name': 'Lat Pulldown Machine', 'price': 1800.00},
        {'name': 'Multipower Machine', 'price': 2600.00}
    ],
    'Accessories': [
        {'name': 'Yoga Mat', 'price': 50.00},
        {'name': 'Resistance Bands', 'price': 30.00},
        {'name': 'Foam Roller', 'price': 40.00},
        {'name': 'Jump Rope', 'price': 20.00},
        {'name': 'Medicine Ball', 'price': 60.00},
        {'name': 'Weight Plates', 'price': 100.00},
        {'name': 'Kettlebell', 'price': 80.00},
        {'name': 'Gym Gloves', 'price': 25.00},
        {'name': 'Resistance Tubes', 'price': 45.00},
        {'name': 'Aerobic Stepper', 'price': 90.00}
    ]
}

# Traduzione dei metodi di pagamento
payment_methods = ['Credit Card', 'Bank Transfer', 'Cash']

def generate_gym_sales(n):
    for _ in range(n):
        category = random.choices(
            list(products.keys()),
            weights=[40, 35, 25],  # Più vendite per Cardio e Strength
            k=1
        )[0]
        product_info = random.choices(
            products[category],
            weights=[10, 9, 8, 7, 6, 5, 5, 5, 5, 5],  # Alcuni prodotti vendono di più
            k=1
        )[0]
        product_name = product_info['name']
        original_price = product_info['price']

        retailer_id = random.choices(
            retailers_id,
            weights=[15] * 5 + [5] * 15,  # 5 retailer con molte vendite, altri meno
            k=1
        )[0]

        customer_id = random.choices(
            customers_id,
            weights=[10] * 30 + [5] * 120,  # 30 clienti più fedeli
            k=1
        )[0]

        quantity = random.choices([1, 2, 3, 4, 5], weights=[70, 15, 10, 3, 2], k=1)[0]

        discount_percentage = random.choice([0, 5, 10, 15, 20, 25])
        discount_amount = round(original_price * (discount_percentage / 100), 2)
        final_price = round((original_price - discount_amount) * quantity, 2)

        sale_date = fake.date_between(start_date="-1y", end_date="today")
        payment_method = random.choice(payment_methods)
        transaction_id = str(uuid.uuid4())[:8]

        # Inseriamo i dati direttamente nel database
        cursor.execute("""
    INSERT INTO sales (
        sale_date, retailer_id, customer_id,
        product_category, product_name, serial_number,
        quantity, unit_price, discount, total_price
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", (
    sale_date, retailer_id, customer_id,
    category, product_name, transaction_id,
    quantity, original_price, discount_percentage, final_price
))

        # Aggiorniamo il contatore dei prodotti più venduti
        # product_trend[product_name] += quantity

    # Commit per salvare le transazioni
    conn.commit()

# Generiamo 5000 vendite per simulare un anno di attività
generate_gym_sales(5000)


print("✅ Sells generated!")

✅ Sells generated!


In [ ]:
query = "SELECT * FROM sales LIMIT 20;"
cursor.execute(query)
result = cursor.fetchall()
for row in result:
  print(row)


(1, datetime.date(2024, 10, 23), 1, 25, 'Strength', 'Leg Press Machine', '5c83d942', 2, Decimal('2000.00'), Decimal('10.00'), Decimal('3600.00'))
(2, datetime.date(2024, 9, 2), 20, 73, 'Strength', 'Olympic Barbell', 'e7f80160', 1, Decimal('350.00'), Decimal('5.00'), Decimal('332.50'))
(3, datetime.date(2024, 4, 19), 1, 38, 'Accessories', 'Kettlebell', 'cc9d26a2', 1, Decimal('80.00'), Decimal('0.00'), Decimal('80.00'))
(4, datetime.date(2024, 7, 8), 10, 113, 'Strength', 'Power Rack', '652b5152', 4, Decimal('1200.00'), Decimal('15.00'), Decimal('4080.00'))
(5, datetime.date(2025, 3, 6), 5, 127, 'Cardio', 'Rowing Machine', '862b41c2', 1, Decimal('1800.00'), Decimal('25.00'), Decimal('1350.00'))
(6, datetime.date(2024, 4, 19), 5, 77, 'Accessories', 'Weight Plates', '4359cfe0', 2, Decimal('100.00'), Decimal('10.00'), Decimal('180.00'))
(7, datetime.date(2025, 2, 10), 10, 23, 'Strength', 'Power Rack', '1968a79a', 1, Decimal('1200.00'), Decimal('5.00'), Decimal('1140.00'))
(8, datetime.date(2

##Expenses Data

In [ ]:
expense_categories = [
    'Rent', 'Salaries', 'Marketing', 'Electricity & Utilities',
    'Equipment Maintenance', 'Insurance', 'Software Licenses', 'Office Supplies'
]

for _ in range(50):  # Generiamo 50 spese
    expense_date = fake.date_between(start_date="-1y", end_date="today")
    category = random.choice(expense_categories)
    amount = round(random.uniform(500, 15000), 2)
    description = f"{category} expense for {fake.month_name()} {fake.year()}"

    cursor.execute("""
    INSERT INTO operating_expenses (
        category, amount, expense_date, description
    ) VALUES (%s, %s, %s, %s)
""", (
    category, amount, expense_date, description
))

conn.commit()
print("✅ Expenses generated!")

✅ Expenses generated!


Checking if the table was correctly populated



In [ ]:
query = "SELECT * FROM operating_expenses LIMIT 10;"
cursor.execute(query)
result = cursor.fetchall()
for row in result:
  print(row)

#CLosing Connection
cursor.close()
conn.close()

(51, 'Software Licenses', Decimal('9481.63'), datetime.date(2025, 2, 5), 'Software Licenses expense for May 1982')
(52, 'Salaries', Decimal('783.36'), datetime.date(2024, 9, 15), 'Salaries expense for December 2014')
(53, 'Salaries', Decimal('13187.70'), datetime.date(2024, 12, 27), 'Salaries expense for April 2022')
(54, 'Salaries', Decimal('5026.18'), datetime.date(2024, 11, 27), 'Salaries expense for September 1996')
(55, 'Software Licenses', Decimal('6932.70'), datetime.date(2024, 8, 20), 'Software Licenses expense for September 1983')
(56, 'Office Supplies', Decimal('5995.71'), datetime.date(2024, 9, 24), 'Office Supplies expense for April 2011')
(57, 'Marketing', Decimal('3814.42'), datetime.date(2024, 9, 16), 'Marketing expense for December 2024')
(58, 'Insurance', Decimal('9934.55'), datetime.date(2024, 8, 13), 'Insurance expense for June 2002')
(59, 'Insurance', Decimal('8536.61'), datetime.date(2024, 9, 25), 'Insurance expense for August 2018')
(60, 'Insurance', Decimal('4210